In [9]:
import yfinance as yf
import pandas as pd
import numpy as np

ml_df = yf.download('^GSPC', start="1978-01-01", interval="1d")
ml_df

/Users/alden/quantfin/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1978-01-03,0.000000,95.150002,93.489998,93.820000,93.820000,17720000
1978-01-04,0.000000,94.099998,92.570000,93.519997,93.519997,24090000
1978-01-05,0.000000,94.529999,92.510002,92.739998,92.739998,23570000
1978-01-06,0.000000,92.660004,91.050003,91.620003,91.620003,26150000
1978-01-09,0.000000,91.480003,89.970001,90.639999,90.639999,27990000
...,...,...,...,...,...,...
2024-03-05,5110.520020,5114.540039,5056.819824,5078.649902,5078.649902,4418410000
2024-03-06,5108.029785,5127.970215,5092.220215,5104.759766,5104.759766,4559050000
2024-03-07,5132.379883,5165.620117,5128.209961,5157.359863,5157.359863,4137980000


In [165]:
sp500_regimes = pd.read_csv('sp500_regimes.csv')
sp500_regimes.set_index('Date', inplace=True)
sp500_regimes.index = pd.to_datetime(sp500_regimes.index)
ml_df.index = pd.to_datetime(ml_df.index)
data = pd.merge(ml_df.drop(columns=['High','Low','Open','Close','Adj Close']), sp500_regimes, on='Date',how='inner')

In [166]:
data['color'] = data['color'].shift(-1)
data.dropna(inplace=True)

In [167]:
X = data.drop(columns=['color'])
y = data['color']

In [168]:
X.shape

(10657, 10)

In [169]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Assume y is your label data
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y, num_classes=4)

In [170]:
# Assume X and y are your data and labels
train_size = int(len(X) * 0.8)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [171]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

# Assuming X is your dataset and y is the target variable
oversampler = SMOTE()  # Choose your oversampler: RandomOverSampler, SMOTE, or ADASYN
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

In [178]:
n_timesteps = 10


# Assuming you have X_resampled_reshaped and y_train as your training data
train_size = int(len(X_resampled) * 0.8)  # 80% for training
X_train, X_val = X_resampled[:train_size], X_resampled[train_size:]
y_train, y_val = y_resampled[:train_size], y_resampled[train_size:]

X_train= np.reshape(X_train,(X_train.shape[0], n_timesteps, X_train.shape[1]//n_timesteps))
X_val = np.reshape(X_val,(X_val.shape[0], n_timesteps, X_val.shape[1]//n_timesteps))
X_test_reshaped = np.reshape(X_test,(X_test.shape[0], n_timesteps, X_test.shape[1]//n_timesteps))

In [179]:
y_resampled.shape

(23468, 4)

In [187]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam


# Assuming your data has 100 time steps and 10 features

n_features = X_train.shape[2]

# Create a Sequential model
model = Sequential()

model.add(LSTM(83, input_shape=(10, 1)))
model.add(Dense(4, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])


model.fit(X_train, y_train, epochs=100, batch_size=32,validation_data=(X_val, y_val))

Epoch 1/100
587/587 [==============================] - 4s 5ms/step - loss: 0.7627 - accuracy: 0.6574 - val_loss: 1.9081 - val_accuracy: 0.0000e+00
Epoch 2/100
587/587 [==============================] - 2s 4ms/step - loss: 0.4780 - accuracy: 0.8019 - val_loss: 1.6087 - val_accuracy: 0.0000e+00
Epoch 3/100
587/587 [==============================] - 2s 4ms/step - loss: 0.3650 - accuracy: 0.8564 - val_loss: 1.1227 - val_accuracy: 0.1536
Epoch 4/100
587/587 [==============================] - 2s 4ms/step - loss: 0.3196 - accuracy: 0.8757 - val_loss: 1.1786 - val_accuracy: 0.2028
Epoch 5/100
587/587 [==============================] - 2s 4ms/step - loss: 0.2400 - accuracy: 0.9112 - val_loss: 0.4263 - val_accuracy: 0.8057
Epoch 6/100
587/587 [==============================] - 2s 4ms/step - loss: 0.1532 - accuracy: 0.9496 - val_loss: 0.1526 - val_accuracy: 0.9540
Epoch 7/100
587/587 [==============================] - 2s 4ms/step - loss: 0.1143 - accuracy: 0.9630 - val_loss: 0.5586 - val_accuracy

In [181]:
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=0)

print(f'Test accuracy: {test_accuracy*100}%')

Test accuracy: 82.59850144386292%


In [183]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from bayes_opt import BayesianOptimization

# Define a function to build and compile the model
def create_model(lr, neurons):
    model = Sequential()
    model.add(LSTM(int(neurons), input_shape=(10, 1)))
    model.add(Dense(4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])
    return model

# Define a function for the hyperparameter optimization
def optimize_model(lr, neurons):
    model = create_model(lr, neurons)
    # Assuming you have X_train, y_train as your training data
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    # Assuming you have X_val, y_val as your validation data
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    return val_accuracy

# Define the bounds of the hyperparameters to optimize
bounds = {'lr': (0.0001, 0.01), 'neurons': (10, 100)}

optimizer = BayesianOptimization(
    f=optimize_model,
    pbounds=bounds,
    random_state=1,
)

optimizer.maximize(init_points=10, n_iter=50)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

|   iter    |  target   |    lr     |  neurons  |
-------------------------------------------------
| 1         | 0.8507    | 0.004229  | 74.83     |


| 2         | 0.2049    | 0.0001011 | 37.21     |


| 3         | 0.02791   | 0.001553  | 18.31     |


| 4         | 0.5684    | 0.001944  | 41.1      |


| 5         | 0.9723    | 0.004028  | 58.49     |


...

| 56        | 0.2009    | 0.009587  | 46.0      |


| 57        | 0.9329    | 0.0014    | 87.76     |


| 58        | 0.0       | 0.006921  | 11.48     |


| 59        | 0.9139    | 0.006773  | 96.74     |


| 60        | 0.601     | 0.00202   | 26.77     |
Iteration 0: 
	{'target': 0.8506604433059692, 'params': {'lr': 0.004228517846555483, 'neurons': 74.82920440979423}}
Iteration 1: 
	{'target': 0.2049424797296524, 'params': {'lr': 0.00010113231069171439, 'neurons': 37.20993153686558}}
Iteration 2: 
	{'target': 0.027907967567443848, 'params': {'lr': 0.0015528833190894193, 'neurons': 18.310473529191803}}
Iteration 3: 
	{'target': 0.5683851838111877, 'params': {'lr': 0.0019439760926389421, 'neurons': 41.1004654338743}}
Iteration 4: 
	{'target': 0.9723050594329834, 'params': {'lr': 0.004027997994883633, 'neurons': 58.493506060302124}}
Iteration 5: 
	{'target': 0.9328930377960205, 'params': {'lr': 0.004250025692592619, 'neurons': 71.66975503570836}}
Iteration 6: 
	{'target': 0.8730294108390808, 'params': {'lr': 0.0021240772723420225, 'neurons': 89.03056927518509}}
Iteration 7: 
	{'target': 0.9748615026473999, 'params': {'lr': 0.00037113717265946903, 'neurons': 70.3420759160562}}
Iteration 8: 